In [1]:
# try making a database for the citations


# index by year


# compare lookup speeds compared to original

In [1]:

# play with some sql selections to make sure things are working

import torch

import time
import mysql.connector as mysql

from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import Row, StructType, StructField, IntegerType, StringType

import os
import time
import random


In [2]:
SUBMIT_ARGS = "--driver-class-path file:///home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar --jars file:///home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = SUBMIT_ARGS  # for java connector

db_name = 'test_pubmed'  # db name collisons? https://stackoverflow.com/questions/14011968/user-cant-access-a-database
table_name = 'citations' # 'abstracts'

url = "jdbc:mysql://localhost:3306/{}?useUnicode=true&useJDBCCompliantTimezoneShift=true&useLegacyDatetimeCode=false&serverTimezone=America/Chicago".format(db_name)  # mysql runs on port 3306


data_dir = '/project2/jevans/brendan/open_citation/'  # this is 2019
open_cite_path = data_dir + 'open_citation_collection_2019-04.csv'
metadata_path = data_dir + 'icite_metadata_2019-04.csv'


client_config = {'unix_socket':'/home/brendanchambers/.sql.sock',
                'database': db_name}  # for python connector



In [3]:
print('initializing spark')
# init spark
conf = SparkConf()
conf = (conf.setMaster('local[*]')
       .set('spark.driver.memory','24G')
       .set("spark.jars", "/home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar"))        
'''
.set('spark.executor.memory','1G')  # 20
.set('spark.driver.memory','1G')   # 40
.set('spark.driver.maxResultSize','500M')  #.set('spark.storage.memoryFraction',0))  # this setting is now a legacy option
.set('spark.python.worker.reuse', 'false')
.set('spark.python.worker.memory','512m')
.set('spark.executor.cores','1'))
'''
sc = SparkContext(conf=conf)
#sc.addJar('home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar')  # temp
spark = SparkSession(sc)  # don't need this for vanilla RDDs

print(sc._conf.getAll())

initializing spark
[('spark.driver.memory', '24G'), ('spark.repl.local.jars', 'file:///home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar'), ('spark.jars', '/home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar'), ('spark.executor.id', 'driver'), ('spark.app.name', 'pyspark-shell'), ('spark.driver.port', '39408'), ('spark.rdd.compress', 'True'), ('spark.driver.extraClassPath', 'file:///home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar'), ('spark.serializer.objectStreamReset', '100'), ('spark.driver.host', 'midway2-0385.rcc.local'), ('spark.master', 'local[*]'), ('spark.app.id', 'local-1565020824064'), ('spark.submit.deployMode', 'client'), ('spark.ui.showConsoleProgress', 'true')]


In [9]:
# load in open citation dataset (citations)

edgelist_schema = StructType(
                        [StructField('citing', IntegerType(), True),
                        StructField('referenced', IntegerType(), True)])


edgelist = spark.read.csv(open_cite_path, 
                          header=True,
                          schema=edgelist_schema)

edgelist.printSchema()

root
 |-- citing: integer (nullable = true)
 |-- referenced: integer (nullable = true)



In [8]:
# load in open citation dataset (metadata)

metadata_schema = StructType([StructField('pmid', IntegerType(), True),
                              StructField('year', IntegerType(), True),
                              StructField('human', StringType(), True),
                              StructField('animal', StringType(), True),
                              StructField('molcel', StringType(), True),
                              StructField('xcoord', StringType(), True),
                              StructField('ycoord', StringType(), True),
                              StructField('approx_potential_translate',StringType(), True),
                              StructField('cited_by_clin', StringType(), True),
                              StructField('title', StringType(), True),
                              StructField('authors', StringType(), True),
                              StructField('journal', StringType(), True),
                              StructField('article', StringType(), True),
                              StructField('is_clinical',StringType(), True),
                              StructField('estimated',StringType(), True),
                              StructField('relative_citation_ratio', StringType(), True)
])
metadata = spark.read.csv(metadata_path,
                          header = True,
                          schema = metadata_schema)



# see schema
metadata.printSchema()

#print(edgelist.count())


root
 |-- pmid: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- human: string (nullable = true)
 |-- animal: string (nullable = true)
 |-- molcel: string (nullable = true)
 |-- xcoord: string (nullable = true)
 |-- ycoord: string (nullable = true)
 |-- approx_potential_translate: string (nullable = true)
 |-- cited_by_clin: string (nullable = true)
 |-- title: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- journal: string (nullable = true)
 |-- article: string (nullable = true)
 |-- is_clinical: string (nullable = true)
 |-- estimated: string (nullable = true)
 |-- relative_citation_ratio: string (nullable = true)



In [10]:
# create citations table
#  note - no need to run this again, took like 2.5 hrs
start_time = time.time()

citations_rdd = edgelist.write.format('jdbc').options(
                                    numPartitions=10,
                                    rewriteBatchedStatements=False,
                                    url=url,
                                    dbtable='citations').mode('overwrite').save()
end_time = time.time()
print("elapsed time: ".format(end_time - start_time))


elapsed time: 


In [9]:
# create metadata table
start_time = time.time()
metadata.write.format('jdbc').options(
                                    numPartitions=4,
                                    rewriteBatchedStatements=False,
                                    url=url,
                                    dbtable='metadata').mode('overwrite').save()
end_time = time.time()
print("elapsed time: ".format(end_time - start_time))

elapsed time: 


In [5]:


db = mysql.connect(**client_config)
cursor = db.cursor()
cursor.close()

sql = "USE test_pubmed;"
cursor = db.cursor()
cursor.execute(sql)  # , multi=True)
cursor.close()

sql = "DESCRIBE citations;"
cursor = db.cursor()
cursor.execute(sql) # , multi=True)
output = cursor.fetchall()
for row in output:
    print(row)
cursor.close()

start_time = time.time()
sql = "SELECT COUNT(*) FROM citations"
cursor = db.cursor()
cursor.execute(sql)
output = cursor.fetchall()
print(output)
end_time = time.time()
print("elapsed: {}".format(end_time - start_time))
cursor.close()

db.close()

('pmid', 'int(11)', 'YES', '', None, '')
('year', 'int(11)', 'YES', '', None, '')
('human', 'text', 'YES', '', None, '')
('animal', 'text', 'YES', '', None, '')
('molcel', 'text', 'YES', '', None, '')
('xcoord', 'text', 'YES', '', None, '')
('ycoord', 'text', 'YES', '', None, '')
('approx_potential_translate', 'text', 'YES', '', None, '')
('cited_by_clin', 'text', 'YES', '', None, '')
('title', 'text', 'YES', '', None, '')
('authors', 'text', 'YES', '', None, '')
('journal', 'text', 'YES', '', None, '')
('article', 'text', 'YES', '', None, '')
('is_clinical', 'text', 'YES', '', None, '')
('estimated', 'text', 'YES', '', None, '')
('relative_citation_ratio', 'text', 'YES', '', None, '')
[(29590934,)]
elapsed: 53.58090591430664


In [18]:
# get baseline timing


#   verify number of rows

#   what is the primary key?   is there one?   check the schema
#    currently the columns are BLOBs instead of unsigned int
#    todo check the range of the pmids


single_pmid = '10037852'  # are these text or ints

#  try to select citing pmid   rows
sql = "SELECT * FROM citations WHERE citing = {}".format(single_pmid)
db = mysql.connect(**client_config)
cursor = db.cursor()

start_time = time.time()
cursor.execute(sql)
output = cursor.fetchall()
print('elapsed time: {}'.format(time.time() - start_time))
for row in output:
    print(row)
cursor.close()

# try to select referenced pmid  rows
sql = "SELECT * FROM citations WHERE referenced = {}".format(single_pmid)
cursor = db.cursor()

start_time = time.time()
cursor.execute(sql)
output = cursor.fetchall()
print('elapsed time: {}'.format(time.time() - start_time))
for row in output:
    print(row)
cursor.close()

db.close()

elapsed time: 0.0006487369537353516
(10037852, 10031166)
(10037852, 10033149)
(10037852, 10033898)
(10037852, 9935586)
(10037852, 9897399)
(10037852, 9897220)
(10037852, 9898169)
(10037852, 10033221)
elapsed time: 0.12197589874267578
(10039315, 10037852)
(10039316, 10037852)
(10045892, 10037852)
(9901154, 10037852)
(9902627, 10037852)
(9906101, 10037852)
(9907020, 10037852)
(9909798, 10037852)
(9965160, 10037852)


In [11]:
# add index  (todo should add all indices at the same time)
#   https://dba.stackexchange.com/questions/141192/mysql-create-index-slowing-down
sql = 'CREATE INDEX citing_index ON citations(citing);'  # 20 min
cursor = db.cursor()
start_time = time.time()
cursor.execute(sql)
end_time = time.time()
print('elapsed time: {}'.format(end_time - start_time))
cursor.close()

db.close()

elapsed time: 1370.728592157364


In [12]:
# index the referenced pmids
sql = 'CREATE INDEX referenced_index ON citations(referenced);'
db = mysql.connect(**client_config)
cursor = db.cursor()
start_time = time.time()
cursor.execute(sql)
end_time = time.time()
print('elapsed time: {}'.format(end_time - start_time))
cursor.close()

db.close()


elapsed time: 1465.5104358196259


In [14]:
# describe again after adding index

db = mysql.connect(**client_config)
cursor = db.cursor()

sql = "DESCRIBE citations;"
cursor.execute(sql)
output = cursor.fetchall()
for row in output:
    print(row)
cursor.close()

db.close()

('citing', 'int(11)', 'YES', 'MUL', None, '')
('referenced', 'int(11)', 'YES', 'MUL', None, '')


In [ ]:
# index metadata and abstracts by year

In [7]:
# add index for year column  in metadata
sql = 'CREATE INDEX citing_index ON metadata(year);'  # 20 min
db = mysql.connect(**client_config)
cursor = db.cursor()
start_time = time.time()
cursor.execute(sql)
print('elapsed time: {}'.format(time.time() - start_time))

elapsed time: 110.54393315315247


In [19]:
sc.stop()